In [36]:
import os
import sys
import copy
from queue import SimpleQueue
from typing import List, Tuple, Dict
from sage.graphs.connectivity import connected_components

Util

In [37]:
def generate_transposition(n: int, u: int, v: int) -> Permutation:
    action = list(range(1, n + 1))
    action[u - 1] = v
    action[v - 1] = u
    return Permutation(action)

generate_transposition(4, 2, 4)  # [1, 4, 3, 2]

[1, 4, 3, 2]

In [38]:
indent = 0


def iprint(s: str, pre: int = 0, post: int = 0):
    global indent
    indent += pre
    print("    " * indent + s)
    indent += post

Input verification

In [39]:
def validate_edges(n: int, allowed_pairs: Tuple[Tuple[int, int], ...]):
    """Checks for invalid edges, duplicate edges (up to ordering), and self-loops."""
    elements = [u for pair in allowed_pairs for u in pair]
    if not (1 <= min(elements) and max(elements) <= n):
        print("Element out of range")
        return False

    for (u, v) in allowed_pairs:
        if u == v:
            print(f"Self-loop found: {(u, v)}")
            return False

    pairs = set([(min(u, v), max(u, v)) for (u, v) in allowed_pairs])
    if len(pairs) != len(allowed_pairs):
        iprint("Duplicate edge found.")
        return False

    if not Graph([list(range(1, n + 1)), list(allowed_pairs)]).is_connected():
        print("Graph is not connected")
        return False

    return True

CC = connected component

`dp[rank of permutation, CC's] = (min length, number of factorizations with such length)`

In [58]:
Edge = Tuple[int, int]
State = Tuple[int, Tuple[Tuple[int, ...], ...]]

def merge_cc(cc_list, u: int, v: int):
    cc_u = [cc for cc in cc_list if u in cc]
    cc_v = [cc for cc in cc_list if v in cc]
    assert len(cc_u) == 1 and len(cc_v) == 1
    cc_u = cc_u[0]
    cc_v = cc_v[0]
    
    if cc_u == cc_v:
        return cc_list
    new_cc_list = list(cc_list)
    new_cc_list.remove(cc_u)
    new_cc_list.remove(cc_v)
    new_cc = tuple(sorted(cc_u + cc_v))
    new_cc_list.append(new_cc)
    new_cc_list.sort()
    return tuple(map(tuple, new_cc_list))


def get_next_states(n, allowed_edges, state):
    next_states = []
    rank, cc_list = state
    perm = Permutations(n).unrank(rank)
    for (u, v) in allowed_edges:
        action = generate_transposition(n, u, v)
        new_perm = Permutation(perm).left_action_product(action)
        new_cc = merge_cc(cc_list, u, v)
        next_states.append((new_perm.rank(), new_cc))
    return next_states


def compute_dp(n: int, allowed_edges: List[Edge]) -> Dict[State, Tuple[int, int]]:
    completed_perms = set()
    base_case = (0, tuple((i,) for i in range(1, n + 1)))
    dp = {base_case: (0, 1)}
    queue = SimpleQueue()
    queue.put(base_case)
    
    while True:
        assert not queue.empty()
        state = queue.get()
        rank, cc_list = state
        perm = Permutations(n).unrank(rank)
        min_length, count = dp[state]

        next_states = get_next_states(n, allowed_edges, state)
        for ns in next_states:
            if ns not in dp:
                dp[ns] = (min_length + 1, count)
                queue.put(ns)
            else:
                next_min_length = dp[ns][0]
                assert next_min_length <= min_length + 1
                if next_min_length == min_length + 1:
                    dp[ns] = (dp[ns][0], dp[ns][1] + count)
        
        if len(cc_list) == 1:
            completed_perms.add(perm)

        # Check that we can terminate
        if len(completed_perms) == factorial(n):
            break
    
    return dp


`compute_length_n(n)`: Returns a map from permutations (a list of n ints) to (min length of factorization, number of such factorizations).

In [71]:
def compute_length_n(n: int, allowed_edges: List[Tuple[int, int]]):
    global indent
    indent = 0
    target_cc_list = (tuple(range(1, n + 1)),)

    dp = compute_dp(n, allowed_edges)
    perm_n = Permutations(n)
    result = {perm: dp[(perm.rank(), target_cc_list)] for perm in perm_n}
    return result, dp

For verifying star transpositions

In [110]:
from functools import reduce
from operator import mul

def star_count(perm: List[int]):
    """Return the number of minimal transitive star factorizations of perm."""
    perm = Permutation(perm)
    cycles = perm.to_cycles()
    cycle_lengths = [len(cycle) for cycle in cycles]
    n = len(perm)
    m = len(cycles)
    return factorial(n + m - 2) / factorial(n) * reduce(mul, cycle_lengths)

In [119]:
def main(n: int, allowed_edges: Tuple[Tuple[int, int], ...]):
    if not validate_edges(n, allowed_edges):
        return
    results = compute_length_n(n, allowed_edges)[0]

    # Output to file
    # filename = 'output.txt'
    # with open(filename, 'w', encoding='utf-8') as file:
    #     for perm, (min_length, count) in results.items():
    #         file.write(f"{perm}: {count} (len {min_length})\n")

    # Output to stdout
    for perm, (min_length, count) in results.items():
        print(f"{perm}: {count} (len {min_length})")
        # For star transpositions:
        # expected = star_count(perm)
        # if count != expected:
        #     print(f">>>>>> WARNING: expected {expected}, actual = {count}")
        

In [121]:
n = 3
test_allowed_pairs = [(1, 2), (2, 3)]
main(n, test_allowed_pairs)

[1, 2, 3]: 4 (len 4)
[1, 3, 2]: 2 (len 3)
[2, 1, 3]: 2 (len 3)
[2, 3, 1]: 1 (len 2)
[3, 1, 2]: 1 (len 2)
[3, 2, 1]: 2 (len 3)
